In [6]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import os
import xarray as xr
import rioxarray as rioxr
import numpy as np
import xdem
import geoutils as gu
import warnings


In [7]:
pth='D:\\Key_works\\20231212dh_calculate'
pth_gla='D:\\Key_works\\20231212dh_calculate\\gla\\CGI1'
pth_srtm='D:\\Key_works\\20231212dh_calculate\\srtm'
pth_srtm_fill='D:\\Key_works\\20231212dh_calculate\\srtm_fill'
pth_dh_xnhl=pth+'/xnhl_test/dh_pene_all'

In [8]:
dxt_ix=pth+'/xnhl_test/shp/dxt_group_xn.shp'
df_dxt=gpd.read_file(dxt_ix)
####选取研究区内的dh
group_lst=[]
for i in df_dxt.reg_group.values:
    if i is not None:
        ist=i.split('+')
        for j in ist:
            group_lst.append(j)
group_lst=list(set(group_lst))# 去除重复元素 保证标识的唯一性
len(group_lst)

261

In [9]:
def nc_out_stp1(gla_dh,gla_dh_fill,gla_h,gpi,n_num,error,outnm_nc):
    ########输出topo dh 的netcdf版本#############################
    mytrans=gla_h.coords['spatial_ref'].GeoTransform
    mytrans=[float(i) for i in mytrans.split(' ')]
    myattrs={'transform': (mytrans[1], mytrans[2], mytrans[0], mytrans[4], mytrans[5], mytrans[3]),
             'crs': '+proj=aea +lat_0=0 +lon_0=105 +lat_1=25 +lat_2=47 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs=True',
             'res': (30.0, 30.0),'nodatavals': 'nan','scales': (1.0,),'offsets': (0.0,),
             'topo process group': '%s'%gpi,
             'effective samples':round(float(n_num),4),
             'dh error':round(float(error),4),}
    ds_nc_out=xr.Dataset(data_vars={'topo_dh':(['y','x'],gla_dh.values[0]),
                                    'elevation':(['y','x'],gla_h.values[0]),
                                    'topo_dh_fill':(['y','x'],gla_dh_fill.values[0])
                                   
                                   },
                         coords={'y':gla_dh.y.values,'x':gla_dh.x.values},attrs=myattrs)
    ds_nc_out['topo_dh'].attrs={'long_name':'Elevation difference from Topo DEM and SRTM DEM (viod version, v1)','units':'m'}
    ds_nc_out['topo_dh_fill'].attrs={'long_name':'Filled elevation difference from Topo DEM and SRTM DEM','units':'m','Fill method':'Normalized regional hypsometric interpolation'}
    ds_nc_out['elevation'].attrs={'long_name':'elevation from viod filled SRTM DEM','units':'m a.s.l'}
    ds_nc_out.to_netcdf(outnm_nc)

###处理DH中异常值  flag=1表示处理
##flag=0表示小冰川不处理
def outlier_rm_ip(gla_dh0,gla_h):
    gla_dh=gla_dh0.where((gla_dh0>-150)&(gla_dh0<150))
    ####小冰川不处理，处理之后空值插值拟合有问题
    if gla_dh.count().values<10:
        return gla_dh,0,gla_dh.count().values/gla_h.count().values
    #############################################################
    gla_ma=gla_h.median().values  #物质平衡线高度，RGI中有些是错误的，因此，这里直接计算
    mask=gla_h.values*0+1  #冰川区mask  
    mask_acc=(gla_h.where(gla_h>=gla_ma)).values*0+1###积累区mask
    mask_abla=(gla_h.where(gla_h<gla_ma)).values*0+1 ##消融区mask
    pos1=np.where(gla_h.values>=gla_ma) ###积累区位置
    pos2=np.where(gla_h.values<gla_ma) ###消融区位置
    #############################################################
    dh_abla=gla_dh*mask_abla
    ##################outlier remove  ref. Zhou 2018 RSE
    w=(gla_h.max()-gla_h)/(gla_h.max()-gla_h.min())
    A=abs(dh_abla).max().values*2
    h_delta=A*w**2
    ggs1=gla_dh*1
    msst=(abs(gla_dh)-abs(h_delta)) 
    #h_delta.plot()
    #print(h_delta)
    pps1=np.where(msst.values>0)##剔除区
    gla_dh.values[pps1]=np.nan
    ###################################################
    dh_acc=gla_dh*mask_acc
    dh_abla=gla_dh*mask_abla

    dh_acc_med=dh_acc.median().values
    dh_abla_med=dh_abla.median().values
           
    return gla_dh,1,gla_dh.count().values/gla_h.count().values


'''generate strip positions and interval center by interval'''
def ExtractByStrip(df,strip_sign,outsign,interval):
    #val_min=df[strip_sign].min()
    #val_max=df[strip_sign].max()
    val_min=1000
    val_max=8500
    #print(val_min,val_max)
    #n=int((val_max-val_min)/interval)
    df[outsign]=-1
    x=val_min
    while x<val_max:
        y=x+interval
        if y>=val_max:
            y=val_max+1
        df.loc[((df[strip_sign]>=x)&(df[strip_sign]<y)),outsign] =(x+y)/2.0
        x=y
    return df

In [14]:
effe_lst=[];
rat_flag_lst=[]
for i in group_lst:  
    nmi=pth_dh_xnhl+'/'+i+'_dh_coreg_pene.tif'
    nm_gla=pth_gla+'\\'+i+'_gla.shp'
    nm_dem_fill=pth_srtm_fill+'\\'+i+'_srtm.tif'
    ds_dem=xr.open_dataset(nm_dem_fill)
    ds_dh=xr.open_dataset(nmi,engine='rasterio')
    df_gla=gu.Vector(nm_gla)
    df_error=pd.read_csv(pth+'/xnhl_test/error_glacier/error_%s.csv'%i)
    for gg_code in df_gla['Glc_Code']:
        gg_shp = gu.Vector(df_gla.ds[df_gla.ds["Glc_Code"] == gg_code])
        gg_h=ds_dem.rio.clip(gg_shp.geometry)['band_data']
        gg_dh0=ds_dh.rio.clip(gg_shp.geometry)['band_data']
        gg_dh=gg_dh0.interp(x=gg_h.x.values,y=gg_h.y.values)
        gg_dh_rm,flag,ratt=outlier_rm_ip(gg_dh,gg_h)
        ####################################
        gla_ma=gg_h.median().values  #物质平衡线高度，RGI中有些是错误的，因此，这里直接计算
        mask=gg_h.values*0+1  #冰川区mask  
        mask_acc=(gg_h.where(gg_h>=gla_ma)).values*0+1###积累区mask
        mask_abla=(gg_h.where(gg_h<gla_ma)).values*0+1 ##消融区mask
        acc_rt=(gg_dh_rm*mask_acc).count().values/(gg_h*mask_acc).count().values
        abla_rt=(gg_dh_rm*mask_abla).count().values/(gg_h*mask_abla).count().values
        if (acc_rt>=0.4)&(abla_rt>=0.5):
            #有效冰川  可以进行插值处理
            rtt_flag=1
            ############################
            (gg_h.fillna(-9999)).rio.to_raster('temp.tif')
            ref_h=xdem.DEM('temp.tif')
            gg_shp_index=gg_shp.rasterize(ref_h)
            signal = xdem.volume.get_regional_hypsometric_signal(ddem=gg_dh_rm.values,
                                                                 ref_dem=ref_h.data,
                                                                 glacier_index_map=gg_shp_index,)
            ############
            ddem_filled = xdem.volume.norm_regional_hypsometric_interpolation(
                    voided_ddem=gg_dh_rm, ref_dem=ref_h, glacier_index_map=gg_shp_index, 
                    regional_signal=signal.dropna())  
            ########################
            gg_dh_rm_fill=gg_dh_rm*1
            gg_dh_rm_fill.values=[ddem_filled]

            df_error_gla=df_error[df_error['gla_id']==gg_code]
            outnm_nc=pth+'/xnhl_test/gla_level_dh_final/nc/'+gg_code+'.nc'
            #outnm_tif=pth+'/xnhl_test/gla_level_dh_final/nc_tif/'+gg_code+'.tif'
            try:
                nc_out_stp1(gg_dh_rm,gg_dh_rm_fill,gg_h,i,df_error_gla['effective samples'].values,df_error_gla['dh_error'].values,outnm_nc)
            except:
                nc_out_stp1(gg_dh_rm,gg_dh_rm_fill,gg_h,i,df_error_gla['effective samples'].values[0],df_error_gla['dh_error'].values[0],outnm_nc)
            ###########################################
            effe_lst.append([i,gg_code,gg_dh_rm.mean().values,gg_dh_rm.median().values,
                             (gg_dh_rm*mask_acc).mean().values,(gg_dh_rm*mask_abla).mean().values,
                             gg_dh_rm_fill.mean().values,gg_dh_rm_fill.median().values,
                             (gg_dh_rm_fill*mask_acc).mean().values,(gg_dh_rm_fill*mask_abla).mean().values,
                            ])
            
        else:
            rtt_flag=0
            ##无效冰川，不生产dh
        rat_flag_lst.append([i,gg_code,ratt,acc_rt,abla_rt,rtt_flag])  
        
    print(i,'done!')
    #break

g0231 done!
g0111 done!
g0200 done!
g0265 done!
g0337 done!
g0365 done!
g0185 done!
g0123 done!
g0273 done!
g0199 done!
g0197 done!
g0158 done!
g0150 done!
g0175 done!
g0027 done!
g0334 done!
g0114 done!
g0100 done!
g0173 done!
g0217 done!
g0232 done!
g0151 done!
g0033 done!
g0209 done!
g0349 done!
g0176 done!
g0017 done!
g0061 done!
g0041 done!
g0004 done!
g0142 done!
g0051 done!
g0312 done!
g0240 done!
g0152 done!
g0331 done!
g0011 done!
g0424 done!
g0126 done!
g0120 done!
g0174 done!
g0347 done!
g0196 done!
g0044 done!
g0178 done!
g0329 done!
g0108 done!
g0235 done!
g0224 done!
g0201 done!
g0256 done!
g0190 done!
g0162 done!
g0202 done!
g0259 done!
g0211 done!
g0030 done!
g0062 done!
g0106 done!
g0253 done!
g0335 done!
g0022 done!
g0155 done!
g0109 done!
g0043 done!
g0169 done!
g0166 done!
g0052 done!
g0180 done!
g0286 done!
g0188 done!
g0414 done!
g0278 done!
g0270 done!
g0315 done!
g0136 done!
g0204 done!
g0317 done!
g0034 done!
g0262 done!
g0219 done!
g0345 done!
g0213 done!
g015

In [237]:
dff_effe=pd.DataFrame(effe_lst,
                      columns=['group_id','gla_id','mean','median','mean_acc','mean_abla',
                               'mean_fill','median_fill','mean_fill_acc','mean_fill_abla'])
effe_nm=pth+'/xnhl_test/dh-effe-sta_xnhl.csv'
dff_effe.to_csv(effe_nm,)

In [238]:
dff_effe

,group_id,gla_id,mean,median,mean_acc,mean_abla,mean_fill,median_fill,mean_fill_acc,mean_fill_abla
0,g0018,G101096E33298N,-2.2477363664235055,-4.501034729049512,-6.477638160943147,2.094463705826747,-2.532606404069561,-4.124494294405553,-6.559231598021659,1.5191851973447386
1,g0018,G101108E33292N,1.7055120859696382,1.4564942258694655,2.391291888114324,1.3545383365419552,1.7664486632688579,1.8403107749920027,2.213537529938661,1.3144241329375668
2,g0018,G101093E33294N,-6.184100287433032,-3.108037407906206,-8.328385448673215,-4.013504878815669,-5.655661475976183,-3.263412033998384,-7.636487289328092,-3.656494682871014
3,g0018,G101093E33290N,-12.320571940108392,-8.514097458653362,-13.887114096194361,-10.910683999631022,-11.220769667891624,-9.17825530459772,-11.557833082840348,-10.849738621979077
4,g0018,G101093E33286N,-5.804584990834597,-6.774280173105558,-4.26381972249485,-7.105395012465693,-5.1462419181403005,-5.558762480361313,-3.311638526646453,-6.99367470398026
...,...,...,...,...,...,...,...,...,...,...
14423,g0341,G087667E27839N,-14.305744244040415,-13.877755675423119,-14.621832453321016,-13.964569668943893,-14.547323091119024,-13.791156150933388,-14.600766258843318,-14.493696269897402
14424,g0341,G087727E27847N,-7.46914588281632,-9.174980926666477,-1.908601579676263,-11.397401374389457,-6.562540148286721,-8.538949697103124,-1.7660597981723278,-11.385741837398966
14425,g0341,G087725E27842N,-7.190112791233111,-10.623819344207073,-1.009441812480414,-10.311663790603161,-5.994087978294833,-7.356001129410733,-1.4136050454047535,-10.75490488492074
14426,g0341,G087663E27836N,-5.2157884990193475,-4.1854104648007375,-5.743342886753424,-4.643015163765209,-7.52223914079417,-9.43930445332672,-7.5348090814424,-7.50966920014594


In [240]:
dff_flag=pd.DataFrame(rat_flag_lst,
                      columns=['group_id','gla_id','glacier_ratio','acc_ratio','abla_ratio','flag'])
flag_nm=pth+'/xnhl_test/dh-glacier-flag.csv'
dff_flag.to_csv(flag_nm,)

In [242]:
dff_flag[(dff_flag.acc_ratio>=0.2)&(dff_flag.abla_ratio>=0.3)]

,group_id,gla_id,glacier_ratio,acc_ratio,abla_ratio,flag
0,g0018,G101096E33298N,0.713396,0.720497,0.706250,1
2,g0018,G101108E33292N,0.704026,0.474067,0.936523,1
3,g0018,G101093E33294N,0.755760,0.756881,0.754630,1
4,g0018,G101093E33290N,0.701107,0.633803,0.775194,1
6,g0018,G101093E33286N,0.783972,0.715278,0.853147,1
...,...,...,...,...,...,...
17979,g0341,G087667E27839N,0.898799,0.931507,0.865979,1
17980,g0341,G087727E27847N,0.734722,0.606648,0.863510,1
17982,g0341,G087725E27842N,0.575290,0.378788,0.779528,1
17983,g0341,G087663E27836N,0.760417,0.791667,0.729167,1
